In [1]:
import pandas as pd
import numpy as np
import itertools
import lightgbm as lgbm
import matplotlib.pyplot as plt

from script import *
from feature_engineering import *
import db_operations as dbop
from constants import *
import data_process as dp

import time

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
pd.set_option('display.max_rows',200)
idx = pd.IndexSlice
agg_operations =['mean','median','max','min','std','size']

In [3]:
%time df = pd.read_hdf(os.path.join('datasets','df_20140101.h5'),key='data')
df = df.sort_index()
df.shape
df.columns

Wall time: 4.71 s


(4593560, 64)

Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', 'turnover_rate', 'turnover_rate_f',
       'volume_ratio', 'pe', 'pe_ttm', 'pb', 'ps', 'ps_ttm', 'total_share',
       'float_share', 'free_share', 'total_mv', 'circ_mv', 'pb*pe_ttm',
       'pb*pe', '5ma', '10ma', '20ma', '40ma', '60ma', '120ma', '250ma',
       'close/5ma', 'close/10ma', 'close/20ma', 'close/40ma', 'close/60ma',
       'close/120ma', 'close/250ma', '60ma_pct', '20ma_pct', 'p5max_abs_pct',
       'p10max_abs_pct', '30ma', '30ma_pct', 'close/30ma', 'prev_close',
       'day_pct', 'close0_rank', 'close0_pct', 'prev_pct', 'prev_day_pct',
       'sell_at', 'sell_at0', 'sell_date', 'sell_idx', 'buy_at', 'buy_at0',
       'max', 'max_idx', 'idx', 'is_selled', 'r'],
      dtype='object')

In [9]:
df.columns

Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', 'turnover_rate', 'turnover_rate_f',
       'volume_ratio', 'pe', 'pe_ttm', 'pb', 'ps', 'ps_ttm', 'total_share',
       'float_share', 'free_share', 'total_mv', 'circ_mv', 'pb*pe_ttm',
       'pb*pe', '5ma', '10ma', '20ma', '40ma', '60ma', '120ma', '250ma',
       'close/5ma', 'close/10ma', 'close/20ma', 'close/40ma', 'close/60ma',
       'close/120ma', 'close/250ma', '60ma_pct', '20ma_pct', 'p5max_abs_pct',
       'p10max_abs_pct', '30ma', '30ma_pct', 'close/30ma', 'prev_close',
       'day_pct', 'close0_rank', 'close0_pct', 'prev_pct', 'prev_day_pct',
       'sell_at', 'sell_at0', 'sell_date', 'sell_idx', 'buy_at', 'buy_at0',
       'max', 'max_idx', 'idx', 'is_selled', 'r', '3ma'],
      dtype='object')

In [8]:
min_periods=1
periods = [3]
for k in periods:
    %time df['{:d}ma'.format(k)] = groupby_rolling(df[['close']],level="code",window=k,ops='mean',min_periods=min_periods)

Wall time: 7.34 s


In [10]:
%time df['p3mean_amt'] = groupby_rolling(df[['amt']],level="code",window=3,ops='mean',min_periods=min_periods)

Wall time: 7.81 s


In [19]:
tmp = pd.DataFrame(index=df.index)
tmp[0]=1
%time tmp['idx']=tmp.reset_index('code').groupby('code').expanding()[0].sum()
tmp.head(20)
%time df['idx'] = tmp['idx']

Wall time: 4.53 s


0   idx
code      date               
000001.SZ 2014-01-02  1   1.0
          2014-01-03  1   2.0
          2014-01-06  1   3.0
          2014-01-07  1   4.0
          2014-01-08  1   5.0
          2014-01-09  1   6.0
          2014-01-10  1   7.0
          2014-01-13  1   8.0
          2014-01-14  1   9.0
          2014-01-15  1  10.0
          2014-01-16  1  11.0
          2014-01-17  1  12.0
          2014-01-20  1  13.0
          2014-01-21  1  14.0
          2014-01-22  1  15.0
          2014-01-23  1  16.0
          2014-01-24  1  17.0
          2014-01-27  1  18.0
          2014-01-28  1  19.0
          2014-01-29  1  20.0

Wall time: 20 ms


In [118]:
target_col = 'p3mean_amt'
target_ss = df[target_col].fillna(0)

# target_ix = np.nonzero(df.columns==target_col)[0][0]
# idx_ix = np.nonzero(df.columns=='idx')[0][0]
# target_ix,idx_ix
# df.columns[[target_ix,idx_ix]]


def idxmax_inc_1(x:np.array):
    return np.argmax(np.nan_to_num(x.astype(float)))+1
# df['']df.groupby('code')[target_col].rolling(20).agg(idxmax_inc_1)


for k in [20]:
#     %time df[['p{}max_p3mean_amt'.format(k),'p{}idxmax_p3mean_amt'.format(k)]] = groupby_rolling(df[[target_col]],level="code",window=k,ops=['max',idxmax_inc_1],min_periods=min_periods)
    %time df['p{}max_p3mean_amt'.format(k)] = groupby_rolling(df[[target_col]],level="code",window=k,ops=['max'],min_periods=min_periods)
    %time df['p{}idxmax_p3mean_amt'.format(k)]= target_ss.rolling(k).apply(lambda x:np.argmax(x))
    df_check = pd.DataFrame(index=df.index)
    df_check['mark']=1
    mask = df_check.groupby('code')['mark'].shift(k - 1).isnull()
    df.loc[mask,'p{}idxmax_p3mean_amt'.format(k)]=np.nan

Wall time: 7.36 s


C:\Users\dell-pc\Anaconda3.6\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """Entry point for launching an IPython kernel.


Wall time: 10.6 s


In [119]:
df[['p{}idxmax_p3mean_amt'.format(k),'p{}max_p3mean_amt'.format(k),target_col,'idx']].head(100)

p20idxmax_p3mean_amt  p20max_p3mean_amt  p3mean_amt  \
code      date                                                              
000001.SZ 2014-01-02                   NaN                NaN         NaN   
          2014-01-03                   NaN                NaN         NaN   
          2014-01-06                   NaN                NaN    6.440451   
          2014-01-07                   NaN                NaN    5.766298   
          2014-01-08                   NaN                NaN    5.372314   
          2014-01-09                   NaN                NaN    5.030948   
          2014-01-10                   NaN                NaN    5.219315   
          2014-01-13                   NaN                NaN    5.289112   
          2014-01-14                   NaN                NaN    4.768594   
          2014-01-15                   NaN                NaN    4.422745   
          2014-01-16                   NaN                NaN    3.724920   
          2014-01-17                   NaN                NaN    3.947280   
          2014-01-20                   NaN                NaN    3.955350   
          2014-01-21                   NaN                NaN    3.800520   
          2014-01-22                   NaN                NaN    4.587520   
          2014-01-23                   NaN                NaN    4.888663   
          2014-01-24                   NaN                NaN    5.768454   
          2014-01-27                   NaN                NaN    5.051187   
          2014-01-28                   NaN                NaN    5.073574   
          2014-01-29                   2.0           6.440451    4.387105   
          2014-01-30                   1.0           6.440451    3.801356   
          2014-02-07                   0.0           6.440451    3.881106   
          2014-02-10                  13.0           5.768454    4.909218   
          2014-02-11                  19.0           9.032334    9.032334   
          2014-02-12                  19.0          10.237650   10.237650   
          2014-02-13                  19.0          12.939544   12.939544   
          2014-02-14                  18.0          12.939544   10.047386   
          2014-02-17                  17.0          12.939544    9.700279   
          2014-02-18                  16.0          12.939544    7.099177   
          2014-02-19                  15.0          12.939544    8.685369   
          2014-02-20                  14.0          12.939544    9.323386   
          2014-02-21                  13.0          12.939544    8.854837   
          2014-02-24                  12.0          12.939544    7.431637   
          2014-02-25                  11.0          12.939544    6.406989   
          2014-02-26                  10.0          12.939544    6.051933   
          2014-02-27                   9.0          12.939544    5.621958   
          2014-02-28                   8.0          12.939544    5.228416   
          2014-03-03                   7.0          12.939544    5.285521   
          2014-03-04                   6.0          12.939544    5.129665   
          2014-03-05                   5.0          12.939544    5.402005   
          2014-03-06                   4.0          12.939544    5.715387   
          2014-03-07                   3.0          12.939544    5.564266   
          2014-03-10                   2.0          12.939544    5.538672   
          2014-03-11                   1.0          12.939544    5.198088   
          2014-03-12                   0.0          12.939544    5.207332   
          2014-03-13                   0.0          10.047386    5.526960   
          2014-03-14                   0.0           9.700279    5.541188   
          2014-03-17                   2.0           9.323386    5.062054   
          2014-03-18                   1.0           9.323386    4.348442   
          2014-03-19                   0.0           9.323386    4.028202   
          2014-03-

In [76]:
idxmax_inc_1 = lambda x:np.argmax(np.nan_to_num(x.astype(float)))+1
idxmax_inc_1.__name__

'<lambda>'

In [68]:
np.argmax(np.nan_to_num(x[:,target_ix].astype(float)))
# x[:,target_ix]
f(x)

308

309.0

In [116]:
target_ss = df[target_col].fillna(0)
%time _ = target_ss.groupby('code').rolling(20).apply(lambda x:np.argmax(x))
import time
t0 = time.time()
result = target_ss.rolling(20).apply(lambda x:np.argmax(x))
df_check = pd.DataFrame(index=df.index)
df_check['mark']=1
mask = df_check.groupby('code')['mark'].shift(k - 1).isnull()
result.loc[mask]=np.nan
print(time.time()-t0)


C:\Users\dell-pc\Anaconda3.6\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """Entry point for launching an IPython kernel.


Wall time: 11.5 s


C:\Users\dell-pc\Anaconda3.6\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """


11.123180627822876


In [124]:
df[[target_col,'idx']].head(100).fillna(0).groupby('code').rolling(20).apply(lambda x:len(x.shape))

C:\Users\dell-pc\Anaconda3.6\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """Entry point for launching an IPython kernel.


p3mean_amt  idx
code      code      date                       
000001.SZ 000001.SZ 2014-01-02         NaN  NaN
                    2014-01-03         NaN  NaN
                    2014-01-06         NaN  NaN
                    2014-01-07         NaN  NaN
                    2014-01-08         NaN  NaN
                    2014-01-09         NaN  NaN
                    2014-01-10         NaN  NaN
                    2014-01-13         NaN  NaN
                    2014-01-14         NaN  NaN
                    2014-01-15         NaN  NaN
                    2014-01-16         NaN  NaN
                    2014-01-17         NaN  NaN
                    2014-01-20         NaN  NaN
                    2014-01-21         NaN  NaN
                    2014-01-22         NaN  NaN
                    2014-01-23         NaN  NaN
                    2014-01-24         NaN  NaN
                    2014-01-27         NaN  NaN
                    2014-01-28         NaN  NaN
                    2014-01-29         1.0  1.0
                    2014-01-30         1.0  1.0
                    2014-02-07         1.0  1.0
                    2014-02-10         1.0  1.0
                    2014-02-11         1.0  1.0
                    2014-02-12         1.0  1.0
                    2014-02-13         1.0  1.0
                    2014-02-14         1.0  1.0
                    2014-02-17         1.0  1.0
                    2014-02-18         1.0  1.0
                    2014-02-19         1.0  1.0
                    2014-02-20         1.0  1.0
                    2014-02-21         1.0  1.0
                    2014-02-24         1.0  1.0
                    2014-02-25         1.0  1.0
                    2014-02-26         1.0  1.0
                    2014-02-27         1.0  1.0
                    2014-02-28         1.0  1.0
                    2014-03-03         1.0  1.0
                    2014-03-04         1.0  1.0
                    2014-03-05         1.0  1.0
                    2014-03-06         1.0  1.0
                    2014-03-07         1.0  1.0
                    2014-03-10         1.0  1.0
                    2014-03-11         1.0  1.0
                    2014-03-12         1.0  1.0
                    2014-03-13         1.0  1.0
                    2014-03-14         1.0  1.0
                    2014-03-17         1.0  1.0
                    2014-03-18         1.0  1.0
                    2014-03-19         1.0  1.0
                    2014-03-20         1.0  1.0
                    2014-03-21         1.0  1.0
                    2014-03-24         1.0  1.0
                    2014-03-25         1.0  1.0
                    2014-03-26         1.0  1.0
                    2014-03-27         1.0  1.0
                    2014-03-28         1.0  1.0
                    2014-03-31         1.0  1.0
                    2014-04-01         1.0  1.0
                    2014-04-02         1.0  1.0
                    2014-04-03         1.0  1.0
                    2014-04-04         1.0  1.0
                    2014-04-08         1.0  1.0
                    2014-04-09         1.0  1.0
                    2014-04-10         1.0  1.0
                    2014-04-11         1.0  1.0
                    2014-04-14         1.0  1.0
                    2014-04-15         1.0  1.0
                    2014-04-16         1.0  1.0
                    2014-04-17         1.0  1.0
                    2014-04-18         1.0  1.0
                    2014-04-21         1.0  1.0
                    2014-04-22         1.0  1.0
                    2014-04-23         1.0  1.0
                    2014-04-24         1.0  1.0
                    2014-04-25         1.0  1.0
                    2014-04-28         1.0  1.0
                    2014-04-29         1.0  1.0
                    2014-04-30         1.0  1.0
                    2014-05-05         1.0  1.0
                    2014-05-06         1.0  1.0
                    2014-05-07         1.0  1.0
